## Numerical Analysis \<Final Project\>

권도혁, 공민경

In [2]:
import torch
import pandas as pd

from tqdm import tqdm
from torch_geometric.nn import DimeNet
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader

#### 1. Call Data

In [3]:
train = pd.read_csv("../input/numerical2022fall/train.csv", index_col=0)
test = pd.read_csv("../input/numerical2022fall/test.csv", index_col=0)
train_num_nodes_list = list()
test_num_nodes_list = list()
train_list = list()
test_list = list()

for idx in tqdm(train.index):
    d = torch.load(f"./data/train/{idx}.pt")
    train_list.append(d)
    train_num_nodes_list.append(d.num_nodes)

for idx in tqdm(test.index):
    d = torch.load(f"./data/test/{idx}.pt")
    test_list.append(d)
    test_num_nodes_list.append(d.num_nodes)

100%|██████████| 1217/1217 [00:00<00:00, 1880.10it/s]


#### 2. Model Generation

In [6]:
Model = DimeNet(hidden_channels=128, out_channels=1, num_blocks=6,num_bilinear=8, num_spherical=7, num_radial=6,
                   cutoff=5.0, envelope_exponent=5, num_before_skip=1, num_after_skip=2, num_output_layers=3)

path = '../data/QM9/'
dataset = QM9(path)
Model, datasets = Model.from_qm9_pretrained(path, dataset, 0)
loader = DataLoader(train_list, batch_size=1)

loss_fn = torch.nn.L1Loss()
optimizer = torch.optim.Adam(Model.parameters(), lr=1e-5, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.96)
swa_scheduler = torch.optim.swa_utils.SWALR(optimizer, swa_lr=0.001)

#### 3. Train

In [8]:
step = 0

for epoch in range(10):
  for data in loader:
    tmp_z = data.x.reshape(-1)
    tmp_z = [int(x)+1 for x in tmp_z]
    tmp_z = torch.tensor(tmp_z)
    
    optimizer.zero_grad()
    out = Model(tmp_z, data.pos)
    loss = loss_fn(out.view(-1), data.y)
    loss.backward()
    optimizer.step()

    step += 1
    if step % 1000 == 0:
      scheduler.step()
    swa_scheduler.step()

0, 

#### 4. Test & Submission file

In [9]:
index = []
result = []

for i, data in enumerate(test_list):
  tmp_z = data.x.reshape(-1)
  tmp_z = [int(x)+1 for x in tmp_z]
  tmp_z = torch.tensor(tmp_z)
  out = Model(tmp_z, data.pos)

  index.append("test_"+str(i))
  result.append(out.item())

code = input()

df = pd.DataFrame(index, columns = ['ID'])
df['predicted'] = result

df.to_csv("../Submissions/submission_"+code+".csv", index = False)